In [14]:
from presentation import Presentation
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout, Dense, Dropout, Input, GlobalAveragePooling1D
import tensorflow.keras as keras
import util
import data_pipeline
import pandas as pd
from encoder import Encoder
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
import math

%load_ext autoreload
%autoreload 2
%load_ext tensorboard

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Load Data

We take the IMBD data, and sort the training data according to the length of the sequence. Sorting introduces more uniform batch sizes w.r.t. the sequence length which reduces training time considerably if combined with dynamical padding. Furthermore, we crop sequences beyond 200 tokens. 

In [15]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

# sort training data w.r.t. the sequence length
seq_length = [len(x) for x in x_train]
permuted_indicies = np.argsort(seq_length)
x_train, y_train = x_train[permuted_indicies], y_train[permuted_indicies]

# crop sequences
x_train = [x[:maxlen] for x in x_train]
x_val = [x[:maxlen] for x in x_val]

25000 Training sequences
25000 Validation sequences


## Dynamical Padding

We overwrite the Keras Sequence class to support dynamical padding which pads batches only and therefore reduce sequence length. This speeds up training because Transformers training time growths quadratically with the sequence legth. See also [Michaël Benesty](https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e) contribution for further details.

In [16]:
class DynamicPadding(Sequence):
    '''
    implements batch sampling with dynamic padding. Credits to

    https://towardsdatascience.com/divide-hugging-face-transformers-training-time-by-2-or-more-21bf7129db9q-21bf7129db9e
    '''

    def __init__(self, tokenized_seqs, targets, batch_size):
        self.y = tokenized_seqs
        self.z = targets
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.y) / self.batch_size)
        
    def __getitem__(self, idx):
        batch_y = self.y[idx * self.batch_size:(idx + 1) *
              self.batch_size]
        batch_z = self.z[idx * self.batch_size:(idx + 1) *
              self.batch_size]
        # padding w.r.t. the longest sequence in the batch
        batch_y = pad_sequences(batch_y, padding='post')
        return batch_y, batch_z
    
# dump the data into the Dynamic Padding batch loader
train = DynamicPadding(x_train, y_train, batch_size=64)
test = DynamicPadding(x_val, y_val, batch_size=64)

## Build the Model

Build a Classifier by using a single encoding layer. The architecture is adopted from the official [Keras example](https://keras.io/examples/nlp/text_classification_with_transformer/) by Apoorv Nandan.

In [17]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = Input(shape=(maxlen,))
encoder_embedding = Encoder(vocab_size + 1, maxlen, embed_dim, num_heads, ffn_units=ff_dim, encoders=1)
x = encoder_embedding(inputs)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

## Compile and train model

In [18]:
adam_opt = Adam(0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# save the model after each epoch
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./imbd_model/prst_model_{epoch}',
        save_freq='epoch',
        ),
    tf.keras.callbacks.TensorBoard(
        log_dir='./imbd_logs',
        profile_batch=0, 
        )
]

# define relevant metrics
metrics = [
    tf.keras.metrics.AUC(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall()
]

# compile model
model.compile(
    optimizer=adam_opt,
    loss="binary_crossentropy",
    metrics=metrics,
#     run_eagerly=True
)

history = model.fit(
    train, validation_data=test, callbacks=callbacks, epochs=2, verbose=1
)

Epoch 1/2
391/391 [==============================] - 47s 120ms/step - loss: 0.4275 - auc_1: 0.8834 - precision_1: 0.7869 - recall_1: 0.8054 - val_loss: 0.3347 - val_auc_1: 0.9352 - val_precision_1: 0.8820 - val_recall_1: 0.8158
Epoch 2/2
391/391 [==============================] - 46s 119ms/step - loss: 0.2286 - auc_1: 0.9673 - precision_1: 0.9091 - recall_1: 0.9098 - val_loss: 0.3670 - val_auc_1: 0.9346 - val_precision_1: 0.8028 - val_recall_1: 0.9195
